## Functions

In [13]:
def get_data_sensor(variable,start_date,end_date):
    '''
    Gets sensor data, expects as input the file, the start and end date
    :variable: string
    :start_date, end_date: string format YYYY-MM-DD HH:MM:SS
    :return: pandas dataframe
    '''
    current_dir = os.path.join(directory) 
    
    print(current_dir)
    print(os.listdir(current_dir))
    

    for file in os.listdir(current_dir):
        filename = os.fsdecode(file)
        if filename == variable + r".csv":
            df_data = pd.read_csv(os.fsdecode(os.path.join(current_dir, file)), sep=";")
            df_data = df_data.set_index('Time')                        
            df_data.index = pd.to_datetime(df_data.index, utc=False)
            df_data_out = df_data[start_date:end_date]
    
    return df_data_out


def get_outliers_dataset(variable,start_date,end_date):
    
    '''
    Gets validated data set (real outliers), expects as input the variable, site name, the start and end date
    :param site: string
    :param start_date: string format YYYY-MM-DD HH:MM:SS
    :return: pandas dataframe
    '''    
    current_dir = os.path.join(directory)
    
    for file in os.listdir(current_dir):
        filename = os.fsdecode(file)
        if filename == variable + r".csv":
            df_data = pd.read_csv(os.fsdecode(os.path.join(current_dir, file)), sep=";", names=["Time", "Value"])
                       
            df_data.replace(r'\n', '', regex = True, inplace = True)                                  
            df_data = df_data.set_index('Time')
            
            df_data.index = pd.to_datetime(df_data.index, utc=False)
            
            #remove duplicated values
            df_data = df_data[~df_data.index.duplicated(keep='first')]
            
            df_data = df_data.sort_index()
            
            #df_data = df_data.set_index('Time')
            #df_data.index = pd.to_datetime(df_data.index, utc=False)            
                       
    return df_data


def Shewhart(points,window = 0, nstd = 3):
    
    #window_size --> size of moving window
    #nstd -->  #number of standard deviations to consider for the boundaries
                
    if window !=0:
        mean = points.rolling(window,min_periods=window // 2,center=True).mean()
        std = points.rolling(window,min_periods=window // 2,center=True).std()
        
    else:
        meanX = points.mean()
        stdX = points.std()               
        mean = points.copy()
        std = points.copy()
        mean.iloc[:] = meanX.values[0]       
        std.iloc[:] = stdX.values[0]         
        
    std.iloc[0] = 0 #the first value turns into NaN because of no data
            
    mean_plus_std = mean + nstd*std
    mean_minus_std = mean - nstd*std        
             
    is_outlier = (points.values > mean_plus_std) | (points.values < mean_minus_std)
    outliers = points[is_outlier]
              
    return outliers, mean, mean_plus_std,mean_minus_std, is_outlier
    

def rolling_IQR(points,window = 0,t0 = 1.5):
    '''
    Classifies a point as an outlier using the Inter quartile range from a boxplot. If it deviates
    a set amount of IQR from the median it's classified as an outlier
    :param points: pandas series
    :param window: 
    :param min_periods: Minimum number of observations in window required to have a value (otherwise result is NA).
    :param t0: threshold value: industry standard is 1.5 but it can go until 3
    '''
    if window != 0:
        q1 = points.rolling(window,min_periods=window//2, center=True).quantile(0.25)
        q3 = points.rolling(window,min_periods=window//2, center=True).quantile(0.75)
        
    else:
        q1X = points.quantile(0.25)
        q3X = points.quantile(0.75)
        q1 = points.copy()
        q3 = points.copy()
        q1.iloc[:] = q1X       
        q3.iloc[:] = q3X  
        
        
    IQR = q3 - q1
    lower_range = q1 - (t0 * IQR)
    upper_range = q3 + (t0 * IQR)   
              
    IQRmask = ~points.between(lower_range, upper_range)    
    IQRmask[:window] = False
    
    outliers = points[IQRmask]  
          
    d = {'IQR': IQR, 'outliers': outliers, 'IQRmask': IQRmask, 'lower_range': lower_range, 'upper_range': upper_range }
    df = pd.DataFrame(data=d)           
    
    return IQR, outliers, IQRmask, lower_range, upper_range, df


def z_score_filter2(points, window= 0, t0=3.5):
    '''
    Calculates the z-score for each measurement assuming a normal distribution of values around the mean
    Then classifies it as an outlier based on the threshold value
    pandas: pandas series of values from which to remove outliers
    window: size of window (including the sample; 7 is equal to 3 on either side of value)
    t0: threshold
    '''
    # Make copy so original not edited
    vals = points.copy()    
   
    if window !=0:
        rolling_mean = vals.rolling(window,min_periods=window // 2,center=True).mean()
        rolling_std = vals.rolling(window,min_periods=window // 2,center=True).std()        
        zscore = (vals - rolling_mean)/rolling_std
        
    else:
        rolling_mean = vals.mean()
        rolling_std = vals.std()
        zscore = (vals - rolling_mean)/rolling_std              
    
    #outlier_idx = np.abs(zscore > t0)    
    outlier_idx = ~((zscore > -t0) & (zscore < t0))    

    mean_plus_std = rolling_mean + t0*rolling_std
    mean_minus_std = rolling_mean - t0*rolling_std 
    
    outliers = points[outlier_idx]
           
    return outliers, zscore, rolling_mean, rolling_std,  mean_plus_std, mean_minus_std, outlier_idx



def mod_z_score_filter(points, window= 0, t0=3.5):
    '''
    Calculates the modified z-score for each measurement. It assumes the 0.75th quartile of the standard normal distribution,
    to which the MAD converges to.
    
    '''
    # Make copy so original not edited
    vals = points.copy()    
   
    if window !=0:
        rolling_median = vals.rolling(window,min_periods=window // 2,center=True).median()
        difference=np.abs(rolling_median-vals)
        median_abs_deviation=difference.rolling(window,min_periods=window // 2,center=True).median()        
        mod_zscore =  0.6745*(vals - rolling_median)/median_abs_deviation                       
         
    else:
        rolling_median = vals.median()
        difference=np.abs(rolling_median-vals)
        median_abs_deviation=difference.median()       
        mod_zscore =  0.6745*(vals - rolling_median)/median_abs_deviation            
    
       
    outlier_idx = ~((mod_zscore > -t0) & (mod_zscore < t0))    

    median_plus_std = rolling_median + t0*median_abs_deviation
    median_minus_std = rolling_median - t0*median_abs_deviation 
    
    outliers = points[outlier_idx]
           
    return outliers, mod_zscore, rolling_median, median_abs_deviation,  median_plus_std, median_minus_std, outlier_idx


def EWMA(points,window_size = 0, nstd = 3):
    
    #window_size --> size of moving window
    #nstd -->  #number of standard deviations to consider for the boundaries
    
    
    mean = points.ewm(halflife=window_size, adjust=False).mean()
    std = points.ewm(halflife=window_size, adjust=False).std()
    std.iloc[0] = 0 #the first value turns into NaN because of no data
        
    mean_plus_std = mean + nstd*std
    mean_minus_std = mean - nstd*std      
             
    is_outlier = (points.values > mean_plus_std) | (points.values < mean_minus_std)
    outliers = points[is_outlier]
              
    return outliers, mean, mean_plus_std,mean_minus_std, is_outlier
                

def constvalue(points,length,eps=0):
    '''
    consequitve points are classified as outliers if they dont change more than a defined minimal
    among withi a defined window
    
    eps : minimal amount of change necessary to not be counted as stationary
    length : minimal amount of measurements in a row 
    '''
    current_length = 0
    data = points.copy()
    data.iloc[:] = False
    list_of_stationaries = []       
    
    for i in range(1,len(data)):
        current_value = points.Value.iloc[i]
        past_value = points.Value.iloc[i-1]
                        
        if abs(current_value-past_value) <= eps:            
            current_length += 1
            #display('current_length:',current_length)
            if current_length == length:
                for measurement in list_of_stationaries:
                    data.iloc[measurement] = True
                list_of_stationaries = []
            if current_length >= length:                
                data.iloc[i] = True
            else:
                list_of_stationaries.append(i)
                
        else:
            current_length = 0
            list_of_stationaries = []
            
    outliers = points[data]    
    mask_stationaries =  data   
    
    Conf_flatline =  mask_stationaries.copy()  
    Conf_flatline.replace(False,100, inplace=True)         
    Conf_flatline.replace(True,0, inplace=True)          
    
    #Conf_flatline.to_csv("Conf_flatline.csv",index=False, encoding='utf-8-sig')
      
    return outliers, mask_stationaries, Conf_flatline


def roc2(points,periods, minv,maxv):
    '''
    point classified as an outlier when the rate of change (roc) between two consecutive points or in a specified period
    exceeds the allowed band
    '''        
    
    period_roc = periods
    
    min_roc = minv    
    max_roc = maxv
    
    #roc = points.pct_change(period_roc,fill_method ='ffill')
    roc = 100*points.pct_change(period_roc)  #units in %          
    
    #List of outliers
    is_outlier = ~((roc.Value > min_roc) & (roc.Value < max_roc))
    outliers = points[is_outlier]  
    
    is_outlier = is_outlier.to_frame()   #Convert series to dataframe
            
    return outliers, roc, is_outlier


#Test to detect exceeding values from a defined range 
def physicalrange(points,minv,maxv):
    '''
    point classified as an outlier when the value exceeds the allowed band
    '''     

    min_range = minv
    max_range = maxv
      
    is_outlier = ~((points.Value > min_range) & (points.Value < max_range))
    outliers = points[is_outlier]
         
    is_outlier = is_outlier.to_frame()   #Convert series to dataframe
    
    return outliers, is_outlier



def DataInspection(points,variable):
    '''
    basic data visualization, data distribution and skewness
    '''        
    
    #Plot time serie, density plot (to see distribution) and box plot (prescense of outliers)
    points[variable].iplot(kind='scatter',filename='cufflinks/simple-scatter',mode='lines', size=4,legend=True)
    
    fig, axes = plt.subplots(1, 2,figsize=(15, 3))

    # Density plot
    sns.kdeplot(ax=axes[0],data=points[variable], shade=True)
    axes[0].set_title('density plot')

    #Box plot
    sns.boxplot(ax=axes[1],x=points[variable])
    axes[1].set_title('box plot')   
    
    #Calculation of sumary statistics and Skewness 
    statistics = points[variable].describe()           
    statistics.loc[len(statistics.index)] = [points[variable].skew()]    
    statistics.rename(index={8:'skewness'},inplace=True)
              
    return statistics   


def MethodEvaluation(realoutliers, outliersmethod):

# Evaluation of the methods. Anomalies as flagged as '1'

    cm_test = confusion_matrix(realoutliers, outliersmethod)                     

    #proportion of predictions that the model classified correctly (good for symmetric datasets)
    accuracy = accuracy_score(realoutliers, outliersmethod)

    #proportion of actual positives that was identified correctly (true possitive rate - TPR)
    recall = recall_score(realoutliers, outliersmethod, average=None)

    #proportion of positive identifications that was actually correct
    precision = precision_score(realoutliers, outliersmethod, average=None)

    #harmonic mean of precision and recall
    f1 = f1_score(realoutliers, outliersmethod, average=None)

    ######### Manual calculation #############

    #True negative:  the method correctly predicts the negative class (non-anomalous data as non-anomalous)
    TN = cm_test[0][0]

    #False negative, type 2 error, the method incorrectly predicts the negative class (non-anomalous) when it is actually positive (anomalous)
    FN  = cm_test[1][0]

    #True positive: methods correctly predicts the positive class (anomalous data as anomalous)
    TP  = cm_test[1][1] 

    #False positive. type 1 error, the method incorrectly predicts the positive class (anomalous) when it is actually negative (non-anomalous)
    FP  = cm_test[0][1] 

    # Sensitivity, hit rate, recall, or true positive rate --> What proportion of actual positives was identified correctly
    TPR = TP/(TP+FN)
    # Specificity or true negative rate --> proportion of actual negatives that are correctly identified
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value --> What proportion of positive identifications was actually correct
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # f1
    f1 = 2 * (PPV * TPR) / (PPV + TPR)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)

    #print("recall: ","%.2f" % TPR,"\nspecificity: ", "%.2f" % TNR,"\nprecision: ","%.2f" % PPV, "\nFNR: " ,"%.2f" % FNR)

    return cm_test, TN, FN, TP, FP, f1, ACC 



def MethodEvaluationAll(realoutliers, outliersmethod):

# Evaluation of the methods. Anomalies as flagged as '1'

    cm_test = confusion_matrix(realoutliers, outliersmethod)
    
    display(cm_test)
    
    #Printing confusion matrix
    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ['{0:0.0f}'.format(value) for value in
                cm_test.flatten()]
    group_percentages = ['{0:.2%}'.format(value) for value in
                     cm_test.flatten()/np.sum(cm_test)]
    labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cm_test, annot=labels, fmt='', cmap='Blues')
    plt.xlabel("predicted")
    plt.ylabel("actual")
         
    print (classification_report(realoutliers, outliersmethod))  

    #proportion of predictions that the model classified correctly (good for symmetric datasets)
    accuracy = accuracy_score(realoutliers, outliersmethod)

    #proportion of actual positives that was identified correctly (true possitive rate - TPR)
    recall = recall_score(realoutliers, outliersmethod, average=None)

    #proportion of positive identifications that was actually correct
    precision = precision_score(realoutliers, outliersmethod, average=None)

    #harmonic mean of precision and recall
    f1 = f1_score(realoutliers, outliersmethod, average=None)

    ######### Manual calculation #############

    #True negative:  the method correctly predicts the negative class (non-anomalous data as non-anomalous)
    TN = cm_test[0][0]

    #False negative, type 2 error, the method incorrectly predicts the negative class (non-anomalous) when it is actually positive (anomalous)
    FN  = cm_test[1][0]

    #True positive: methods correctly predicts the positive class (anomalous data as anomalous)
    TP  = cm_test[1][1] 

    #False positive. type 1 error, the method incorrectly predicts the positive class (anomalous) when it is actually negative (non-anomalous)
    FP  = cm_test[0][1] 

    # Sensitivity, hit rate, recall, or true positive rate --> What proportion of actual positives was identified correctly
    TPR = TP/(TP+FN)
    # Specificity or true negative rate --> proportion of actual negatives that are correctly identified
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value --> What proportion of positive identifications was actually correct
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # f1
    f1 = 2 * (PPV * TPR) / (PPV + TPR)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)

    print("recall: ","%.2f" % TPR,"\nspecificity: ", "%.2f" % TNR,"\nprecision: ","%.2f" % PPV, "\nFNR: " ,"%.2f" % FNR)

    return cm_test 



def resampling(points, freq, mod):       
    
    '''
    Downsampling — Resample to a wider time frame. It can use aggregation functions: 
      mean(), min(), max(), sum(), last()-->the last notnull value
    Upsampling — Resample to a shorter time frame. New rows can be filled out with: 
      ffill() --> Use the last known value to fill the new one
      bfill() --> Use the next known value to fill the new one 
    '''

    resampMode = mod
    base = freq
    
    #Check first for duplicated data    
    points = points[~points.index.duplicated(keep='first')]
    
    #Resampling of the data    
    if resampMode == 'last':
        data_res = points.resample(base).last()
        
    elif resampMode == 'bfill':     
        data_res = points.resample(base).bfill()
        
    elif resampMode == 'ffill':     
        data_res = points.resample(base).ffill()  
        
    elif resampMode == 'mean':     
        data_res = points.resample(base).mean() 
        
    elif resampMode == 'min':     
        data_res = points.resample(base).min()
        
    elif resampMode == 'max':     
        data_res = points.resample(base).max()
        
    elif resampMode == 'sum':     
        data_res = points.resample(base).sum() 
               
    elif resampMode == 'default':     
        data_res = points.resample(base) 
                                   
    return data_res


#Functions for ARIMA/ARMA calculations

#Test stationarity of the data using the Augmented Dickey-Fuller (ADF) test
def test_stationarity(ts_data, column='', signif=0.05, series=False):
    if series:
        adf_test = adfuller(ts_data, autolag='AIC')
    else:
        adf_test = adfuller(ts_data[column], autolag='AIC')
    p_value = adf_test[1]
    if p_value <= signif:
        test_result = "Stationary"
    else:
        test_result = "Non-Stationary"
        
    return test_result

#Conver data into stationary
def differencing(data, column, order):
    differenced_data = data[column].diff(order)
    differenced_data.fillna(differenced_data.mean(), inplace=True)
    
    return differenced_data

def get_order_sets(n, n_per_set) -> list:
    n_sets = [i for i in range(n)]
    order_sets = [
        n_sets[i:i + n_per_set]
        for i in range(0, n, n_per_set)]
    
    return order_sets

def find_aic_for_model(data, p, q, model, model_name):
    try:
        msg = f"Fitting {model_name} with order p, q = {p, q}n"
        print(msg)
        if p == 0 and q == 1:
            # since p=0 and q=1 is already
            # calculated
            return None, (p, q)
        ts_results = model(data, order=(p, q)).fit(disp=False)
        curr_aic = ts_results.aic
        return curr_aic, (p, q)
    except Exception as e:
        f"""Exception occurred continuing {e}"""
        return None, (p, q)

#Best order is found using the minimum AIC value for the model
def find_best_order_for_model(data, model, model_name):
    p_ar, q_ma = max_p, max_q
    final_results = []
    warnings.filterwarnings("ignore")
    ts_results = model(data, order=(0, 1)).fit(disp=False)    
    min_aic = ts_results.aic
    final_results.append((min_aic, (0, 1)))
    # example if q_ma is 6
    # order_sets would be [[0, 1, 2, 3, 4], [5]]
    order_sets = get_order_sets(q_ma, 5)
    for p in range(0, p_ar):
        for order_set in order_sets:
            # fit the model and find the aic
            results = Parallel(n_jobs=len(order_set), prefer='threads')(
                delayed(find_aic_for_model)(data, p, q, model, model_name)
                for q in order_set
            )
            final_results.extend(results)
    results_df = pd.DataFrame(final_results,columns=['aic', 'order'])
    
    min_df = results_df[results_df['aic'] == results_df['aic'].min()]
    min_aic = min_df['aic'].iloc[0]
    min_order = min_df['order'].iloc[0]
    
    return min_aic, min_order, results_df

#Find anomalies for ARIMA/ARMA model
def find_anomalies(squared_errors,z):
    threshold = np.mean(squared_errors) + z*np.std(squared_errors)
    predictions = (squared_errors >= threshold).astype(int)
    return predictions, threshold